Notebook inspired by repo: https://github.com/Particionamiento/GraphPartitioningMinSize

## Problem

Let  
* $G = (V,E)$ be an undirected graph, with node set $V = \{1,\ldots, n\}$ and edge set $E \subseteq \{ (i,j) | i,j\in V, i \neq j \}$
* $ w_v: V \to \mathbb{R}_{\geq}$ be the non-negative weights function to the vertices of $G$
* $ w_e: E \to \mathbb{R}_{\geq}$ be the non-negative weights function to the edges of $G$
* $k\geq 2$ an integer denoting the number of connected components to partition $G$ into

Find
* $\mathcal{F}=\{(V_i, E_i)\}_{i\in[k]}$ a connected $k$-partition of $G$  
where $V_i\cap V_j=\emptyset$ for $i \neq j$, $\bigcup_{i=1}^k V_i=V$, and $V_i \neq \emptyset$, for all $ i, j \in [k]$. $\bigcup_{i=1}^k E_i \subseteq V$ for all $ i \in [k]$. $[k] := \{1,\ldots,k\}$

Goal
* minimize $\max_{i \in [k]} \{w(G_i)\}$  
$ w(G) denotes the weight function composed of $w_v$ and $w_e$ for your specific problem

By this definition, $G$ can be a cyclic graph with several vertices zero-weighted. $\mathcal{F}$ is generated from edge cut. With certain mathermatical modeling, especially the following one, $\mathcal{F}$ results in a set of spanning trees. Thus, it can be termed as a spanning forest problem. However, it is not the minimum spanning forest because the optimization does not target the average depth.

## Formulation

Variable
* $f_a$ a non-negative continuous variable representing the flow over the arc $a:u\rightarrow v$
* $y_a$ a binary variable associated with the arc, taking the value of one if $f_a > 0$, and zero otherwise

The model below is inspired by this paper: Partitioning a graph into balanced connected classes: Formulations, separation and experiments, Flávio K. Miyazawa, e.g. (2021). Notions for its construction:
* presume a node set for virtual sources $S = \{s_i\}_{i \in |k|}$ with $k \geq 2$ to construct a directed graph $G'=(V \cup S, A)$ contains all possible flow arcs based on $G$. Scenario 1) with the wire-in nodes for sources undefined: $A = \{(u,v)|u,v \in V, u\neq v\} \cup \{(u,v)|u\in S, v\in V\}$. Scenario 2) with wire-in nodes for sources specified as $O \subset V$: $A = \{(u,v)|u,v \in V, u\neq v\} \cup \{(u,v)|u \in S, v\in O\}$. Either scenario we have sources defined as $S$ and $V$ containing the terminals or relays.
* $\delta^−(v)$ and $\delta^+(v)$ represent the ingoing and outgoing arcs of the node $v$, respectively
* $y(A)=\sum_{a \in A} y_a$ and $f(A)=\sum_{a\in A} f_a$ as a shorthand for summation of variables in a set

\begin{align}
&\min f(\delta^+(s_1))
\\
&\min \sum_{a \in A} {f_a \cdot w_e(a)}
\\
&\min \sum_{v \in V} {\left(\frac {1} {|\delta^- (v)|} \sum_{a \in \delta^- (v)} {\left(f_a - \frac {f(\delta ^ - (v))} {|\delta ^- (v)|} \right) ^2 }\right) ^ {1/2} }
\\
\text{s.t.} & 
\\
&f(\delta^+(s_i)) \geq f(\delta^+(s_{i+1})), &&\forall i \in [k-1],
\\
&f(\delta^-(v)) - f(\delta^+(v))=w_v(v), &&\forall v\in V,
\\
&y(\delta^+(s)) \leq 1, &&\forall s \in S,
\\
&y(\delta^-(v)) \leq 1, &&\forall v \in V,
\\
&f_a \leq w_v(G)y_a, &&\forall a \in A,
\\
&y_a \in \{0, 1\}, &&\forall a \in A,
\\
&f_a \in \mathbb{R}_\geq, &&\forall a \in A.
\end{align}

## Implementation

Gurobi is the solver for this Mixed Integer Programming

In [1]:
import re
import json

import gurobipy as gp
from gurobipy import GRB

import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
with open('graph.json', 'r') as f:
    data = json.load(f)

# Unzip nodes and edges from JSON
node_wgts = [node['weight'] for node in data['nodes']]
sizes = [node['weight']/20 for node in data['nodes']]
edges = [(edge['source'], edge['target']) for edge in data['links']]
edge_wgts = [edge['weight'] for edge in data['links']]
# be careful whether these edges are bi-directional or not

G = nx.Graph()
# initiate nodes before adding edges to prevent disorder
G.add_nodes_from([i for i in range(len(node_wgts))])
G.add_edges_from(edges)
V = list(G.nodes())
E = list(G.edges())
# update the node/edge weight
for i in range(len(node_wgts)):
    G.nodes[i]['weight'] = node_wgts[i]
for i in range(len(edges)):
    G.edges[edges[i][0], edges[i][1]]['weight'] = edge_wgts[i]
print('(V,E) =', (len(V), len(E)))

In [ ]:
# Display graph
plt.figure(figsize=(10,10))
pos = nx.spring_layout(G, seed=25)
nx.draw_networkx_nodes(G, pos, node_color='orange', node_size=sizes, alpha=0.7)
nx.draw_networkx_edges(G, pos, edge_color='gray')
nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif')
plt.title('Connected graph' if nx.is_connected(G) else 'Warning: unconnected graph')
plt.draw()

In [ ]:
# create a directed graph G'=(S ∪ V, A) based on G(V, E)
# append virtual sources S with the cardinality k
# double E to form anti-parallel arcs
# add one-way connection from S to possible wire-in nodes in V
k = 2
S = [len(V) + i for i in range(k)]
u_to_v = E
v_to_u = [(j,i) for (i,j) in E]

# SCENARIO 1
# s_to_v = [(i,j) for i in S for j in V]
# SCENARIO 2
O = [111, 112]
s_to_v = [(i,j) for i in S for j in O]

# # arc weight in line with coresponding edge
A = u_to_v + v_to_u + s_to_v
arc_wgts = [0 for i in range(len(A))]
for i in range(len(E)):
    arc_wgts[2*i] = edge_wgts[i]
    arc_wgts[2*i + 1] = edge_wgts[i]

print(len(A), len(arc_wgts))
Gx = nx.DiGraph()
Gx.add_nodes_from([x for x in range(len(S) + len(V))])
Gx.add_edges_from(A)
nx.set_edge_attributes(Gx, 1, 'weight')
nx.set_node_attributes(Gx, 0, 'weight')
for i, weight in enumerate(node_wgts):
    Gx.nodes[i]['weight'] = weight
for i, arc in enumerate(A):
    Gx.edges[arc[0], arc[1]]['weight'] = arc_wgts[i]
W = sum(node_wgts)

### Create model

In [ ]:
if 'mo' in globals():
    mo.dispose()
    gp.disposeDefaultEnv()
    del mo

In [ ]:
mo = gp.Model()
y, f = gp.tupledict(), gp.tupledict()    # Dictionaries will contain variables

# Binary indicator of flow direction (permitted or not)
y = mo.addVars(A, vtype=GRB.BINARY, name = 'y')
# flow on an arc induced by A
f = mo.addVars(A, vtype=GRB.CONTINUOUS, name = 'f')

# variance on each node for flow balancing
v = mo.addVars(A, lb=-GRB.INFINITY, vtype=GRB.CONTINUOUS, name = 'v')
b = mo.addVars(A, vtype=GRB.CONTINUOUS, name = 'b')

mo.update()

In [12]:
# note that quicksum() takes Var as input
# abs_() returns LinExpr so it cannot be summed up
# path with zero flow will also be counted in the mean value calculation
def GetMean(fs):
    res = gp.quicksum(fs) / len(fs)
    return res

In [ ]:
mo.ModelSense = GRB.MINIMIZE
mo.setObjectiveN(gp.quicksum(f[S[0], v] for v in Gx.successors(S[0])), 2, 0) # load variance
mo.setObjectiveN(gp.quicksum(b[i,j] for (i,j) in A), 1, 1) # flow variance
mo.setObjectiveN(gp.quicksum(f[i,j] * Gx.edges[i,j]['weight'] for (i,j) in A), 0, 2) # unit coverage

# The outflow of s_i follows an ascending order so the optimum function may work
mo.addConstrs((gp.quicksum(f[i,j] for j in Gx.successors(i)) >= gp.quicksum(f[i+1,j] for j in Gx.successors(i+1)) 
               for i in S[0:-1]), name='R1')
# the vertex weight equals to the flow it consumes from inflow
mo.addConstrs((gp.quicksum(f[i,v] for i in Gx.predecessors(v)) - gp.quicksum(f[v,j] for j in Gx.successors(v)) \
               == Gx.nodes[v]['weight'] for v in V), name='R2')
# flow direction restriction
mo.addConstrs((f[i,j] <= W * y[i,j] for (i,j) in A), name='R3')
# ???
mo.addConstrs((y[i,j] + y[j,i] <= 1 for (i,j) in [item for item in A if item not in s_to_v]), name='R3+')
# conditional constraint
# mo.addConstrs(((f[i,j] == 0) >> (y[i,j] == 0) for (i,j) in A), name="R3-")
# all outflows from each source node can only be 0 or 1
mo.addConstrs((gp.quicksum(y[i,j] for j in Gx.successors(i)) <= 1 for i in S), name='R4')
# if you need to restirct the outflow of all nodes to be only one path, change it to this:
# mo.addConstrs((quicksum(y[i,j] for j in Gx.neighbors(i, mode='out')) <= 1 for i in (S + T)), name='R4')
# all inflows from each vertex node can only be 0 or 1
mo.addConstrs((gp.quicksum(y[i,j] for i in Gx.predecessors(j)) <= 1 for j in V), name='R5')


# additional constraints for flow balancing
mo.addConstrs((v[i,j] == f[i,j] - GetMean([f[i,h] for h in Gx.successors(i)]) for (i,j) in A), name='R6')
mo.addConstrs((b[i,j] == gp.abs_(v[i,j]) for (i,j) in A), name='R7')


In [14]:
mo.Params.TimeLimit = 5*60
mo.optimize()
if mo.Status in (GRB.INF_OR_UNBD, GRB.INFEASIBLE, GRB.UNBOUNDED):
    print("The model cannot be solved because it is infeasible or unbounded")
    mo.computeIIS()
    mo.write("model.lp")
if mo.Status != GRB.OPTIMAL:
    print(f"Optimization was stopped with status {mo.Status}")

Set parameter TimeLimit to value 300
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22000.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 933 rows, 944 columns and 2421 nonzeros
Model fingerprint: 0x781802c3
Model has 236 general constraints
Variable types: 708 continuous, 236 integer (236 binary)
Coefficient statistics:
  Matrix range     [3e-01, 1e+05]
  Objective range  [7e-15, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+04]

---------------------------------------------------------------------------
Multi-objectives: starting optimization with 3 objectives ... 
---------------------------------------------------------------------------

Multi-objectives: applying initial presolve ...
---------------------------------------------------------------------------

Presolve added 121 rows and 67 col

In [15]:
# for v in mo.getVars():
#     print(f"{v.VarName} = {v.X}")
x = mo.getVars()
var_keys = f.keys
forest = []
varSolutions = []

nSolutions  = mo.SolCount
nObjectives = mo.NumObj
print('Problem has', nObjectives, 'objectives')
print('Gurobi found', nSolutions, 'solutions')
for s in range(nSolutions):
    # mo.params.SolutionNumber = s
    mo.setParam(GRB.Param.SolutionNumber, s)
    print('Solution', s, ':', end='')
    for o in range(nObjectives):
        # mo.params.ObjNumber = o
        mo.setParam(GRB.Param.ObjNumber, o)
        print(' ',mo.ObjNVal, end='')
    print('')
    
    tree = []
    varSolution = []
    for var in mo.getVars():
        varSolution.append(var.Xn)
        if var.Xn > 0.0:
            result = re.match(r'f\[(\d+),(\d+)\]', var.VarName)
            if result is not None:
                node_start, node_end = result.groups()
                tree.append((int(node_start), int(node_end)))
    forest.append(tree)
    varSolutions.append(varSolution)

Problem has 3 objectives
Gurobi found 5 solutions
Solution 0 :  2281341.628270228  1105973.0768966465  83594.83705998724
Solution 1 :  2281341.628270231  1105973.076896646  83594.83705998724
Solution 2 :  2281299.4147346  1114353.2590520005  84639.910464
Solution 3 :  2856414.1033142908  1245773.2578917237  63684.59583099722
Solution 4 :  3279813.0945201735  1666056.0467397308  116476.06171597913


In [16]:
import copy

# connections = []

# for v in mo.getVars():
#     if v.X > 0.0:
#         result = re.match(r'f\[(\d+),(\d+)\]', v.VarName)
#         if result is None:
#             break
#         node_start, node_end = result.groups()
#         connections.append((int(node_start), int(node_end)))

connections = forest[0]

# here I remove all arcs spreading from source nodes
for i in reversed(range(len(connections))):
    if connections[i][0] in S or connections[i][1] in S:
        connections.remove(connections[i])

# rebuild the graph that has connected vertices
clusters = []
_connections = copy.deepcopy(connections)
def isJoined(edge1, edge2):
    for i in range(2):
        if edge1[i] == edge2[i] and edge1[1-i] != edge2[1-i] or \
           edge1[i] == edge2[1-i] and edge1[1-i] != edge2[i]:
            return True
def flattenClusters(nested_edges):
    nested_nodes = []
    for edges in nested_edges:
        nodes = []
        for edge in edges:
            nodes.append(edge[0])
            nodes.append(edge[1])
        nested_nodes.append(list(set(nodes)))
    return nested_nodes

while len(_connections) > 0:
    flag = True
    while flag:
        flag = False
        for cluster in clusters:
            for edge in cluster:
                for i in reversed(range(len(_connections))):
                    if isJoined(_connections[i], edge):
                        cluster.append(_connections[i])
                        _connections.remove(_connections[i])
                        flag = True
                        break
    if len(_connections) > 0:
        clusters.append([_connections[-1]])
        _connections.remove(_connections[-1])

partition_node = flattenClusters(clusters)
partition_num = len(partition_node)
partition_edge = [[] for i in range(partition_num)]
removed_edge = []
for edge in list(E):
    edge_rev = (edge[1], edge[0])
    if edge in connections or edge_rev in connections:
        for i in range(partition_num):
            if edge[0] in partition_node[i] or edge[1] in partition_node[i]:
                partition_edge[i].append(edge)
    else:
        removed_edge.append(edge)

print("-nodes-", partition_node)
print("-edges-", partition_edge)
print("-del_edge-", removed_edge)
print("number of clusters is ", partition_num)


-nodes- [[18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 39, 40, 51, 52, 53, 54, 55, 59, 62, 63, 66, 67, 68, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 108, 111], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 56, 57, 58, 60, 69, 70, 81, 82, 83, 84, 85, 86, 87, 88, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 112]]
-edges- [[(18, 19), (18, 30), (19, 20), (19, 71), (20, 21), (20, 95), (21, 22), (21, 72), (22, 23), (22, 94), (23, 24), (23, 73), (24, 25), (24, 93), (25, 26), (25, 74), (26, 27), (26, 92), (27, 28), (27, 75), (28, 29), (28, 76), (29, 31), (29, 34), (30, 55), (30, 59), (31, 32), (31, 77), (32, 33), (32, 89), (33, 39), (33, 78), (34, 35), (34, 91), (35, 90), (39, 40), (39, 79), (40, 80), (51, 52), (51, 54), (52, 53), (52, 108), (53, 59), (54, 99), (55, 111), (59, 62), (62, 63), (62, 68), (63, 98), (66, 67), (66, 68), (67, 97), (68, 96)], [(1, 2), (1, 100

In [17]:
size_scheme = []
for partition in partition_node:
    size_scheme.append([sizes[i] for i in partition])
print(size_scheme)

[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 89.1209145, 62.69559965, 62.335128350000005, 62.04938575, 62.18350769999999, 61.98747005, 72.70626250000001, 108.96776385000001, 52.27246720000001, 52.05346335, 45.034707250000004, 44.3868663, 43.19319835, 42.53449955, 43.28402345, 43.43405025, 47.2727534, 159.04147135, 239.33232145000002, 1240.6466898, 1327.00281115, 218.2064978, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 55.472864, 65.7099752, 52.2536702, 53.523082599999995, 95.05499695, 54.01475455, 51.0251412, 53.8783796, 44.3509937, 64.53363315, 68.3901941, 38.010386249999996, 40.11389955, 568.8212267, 64.40667545, 259.66807574999996, 96.92212620000001, 163.88833535, 63.1292916, 39.0360003, 0.0]]


In [ ]:
plt.figure(figsize=(10,10))
pos = nx.spring_layout(G, seed=59)
colors = ['red', 'orange', 'blue', 'green', 'pink']

for i in range(partition_num):
    nx.draw_networkx_nodes(G, pos, nodelist=partition_node[i], node_color=colors[i], node_size=size_scheme[i], alpha=0.7)
# draw source nodes if they are inside G or the graph layout will change
# nx.draw_networkx_nodes(G, pos, nodelist=S, node_color='grey', node_size=100, node_shape='s', alpha=0.5)

for i in range(partition_num):
    nx.draw_networkx_edges(G, pos, edgelist=partition_edge[i], edge_color=colors[i], arrows=None)
nx.draw_networkx_edges(G, pos, edgelist=removed_edge, edge_color='gray', arrows=None)

nx.draw_networkx_labels(G, pos, font_size=10, font_family='sans-serif')
plt.draw()

In [ ]:
Data = pd.DataFrame.from_dict(
    {'Instance':[(len(V),len(E),k)],
     'z_R':[mo.ObjBound],'Obj': mo.objval,'gap': mo.MIPgap,'nodes':int(mo.nodecount), 'time':mo.RunTime})
display(Data)

In [ ]:
# patrol walks

k = 2
Gx = ig.Graph(directed = True)
Gx.add_vertices(10)
S = [8, 9]
V = [0, 1, 2, 3, 4, 5, 6, 7]
edge_list = [(0,1), (1,2), (0,2), (2,3), (3,4), (3,5), (4,5), (4,6), (5,7), (6,7)]
edge_list_rev = [(j,i) for (i,j) in edge_list]
source_edges = [(i,j) for i in S for j in V]
weights = [2, 1, 1, 3, 1, 4, 3, 2]
A = edge_list + edge_list_rev + source_edges
D = edge_list + edge_list_rev
Gx.add_edges(A)
for i, weight in enumerate(weights):
    Gx.vs[i]['weight'] = weight
W = 17